In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
device = 'cuda'

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import sys
import numpy as np
# from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split

sys.path.append("gdrive/My Drive/Year 4: Synthesis/NS162: Statistical Mechanics/Renormalization Research Project/minerva/NS162/supervised_convnet/")

class SupervisedConvNet(nn.Module):
    def __init__(self, filter_size, square_size):
        super(SupervisedConvNet, self).__init__()
        self.filter_size = filter_size
        self.square_size = square_size
        self.conv2d = nn.Conv2d(1, 1, filter_size, padding=0, stride = filter_size)  
        self.linear = nn.Linear(filter_size ** 2, 1)
        

        
    def forward(self, x):
        # add hidden layers with relu activation function
        x = F.relu(self.conv2d(x))
        
        x = x.view(-1, 1, self.square_size)
        x = torch.sigmoid(self.linear(x))
        # for row in x:
        #     print("row", row)
        #     for el in row[0]:
        #         print("el", el)
        # x = torch.tanh(self.decoder(x))
        return x

class IsingDataset(Dataset):
    def __init__(self, data, label):
        self.X = X_train
        self.y = y_train
        
        
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    
    def __len__(self):
        return len(self.X)
   


uncorrelated_data= np.load("gdrive/My Drive/Year 4: Synthesis/NS162: Statistical Mechanics/Renormalization Research Project/minerva/NS162/ising81x81_temp1_uncorrelated9x9.npy")
correlated_data= np.load("gdrive/My Drive/Year 4: Synthesis/NS162: Statistical Mechanics/Renormalization Research Project/minerva/NS162/ising81x81_temp1.npy")[:,:9,:9]
data = np.vstack((uncorrelated_data, correlated_data))
label = np.hstack((np.zeros(10000), np.ones(10000)))
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.33, random_state=42)
isingdataset = IsingDataset(X_train, y_train)


In [0]:
?supervised_convnet

In [0]:
model = SupervisedConvNet(filter_size = 3, square_size = 9)


In [7]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print (name, param.data)

conv2d.weight tensor([[[[-0.6480, -0.4793,  0.5110],
          [-0.2615,  1.0138, -1.3004],
          [ 0.9064,  0.5151,  0.2655]]]])
conv2d.bias tensor([-1.8766])
linear.weight tensor([[-1.2107, -0.0769,  0.1795,  0.0711,  0.1284,  0.1111, -5.9926, -0.4588,
         -0.3866]])
linear.bias tensor([0.0195])


In [6]:
# specify loss function
criterion = nn.BCELoss()

# specify loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

# Create training and test dataloaders
num_workers = 0
# how many samples per batch to load
batch_size = 256
# number of epochs to train the model
n_epochs = 500

# prepare data loaders
train_loader = torch.utils.data.DataLoader(isingdataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)

for epoch in range(1, n_epochs+1):
    # monitor training loss
    train_loss = 0.0

    ###################
    # train the model #
    ###################
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.unsqueeze(1).type('torch.FloatTensor')
        target = target.type('torch.FloatTensor')
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        # update running training loss
        train_loss += loss.item() * batch_size

    # print avg training statistics 
    train_loss = train_loss/len(train_loader)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch, 
        train_loss
        ))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:512: UserWarning: Using a target size (torch.Size([256])) that is different to the input size (torch.Size([256, 1, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:512: UserWarning: Using a target size (torch.Size([88])) that is different to the input size (torch.Size([88, 1, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


Epoch: 1 	Training Loss: 177.330521
Epoch: 2 	Training Loss: 177.321879
Epoch: 3 	Training Loss: 177.304175
Epoch: 4 	Training Loss: 177.331250
Epoch: 5 	Training Loss: 177.335371
Epoch: 6 	Training Loss: 177.316226
Epoch: 7 	Training Loss: 177.324800
Epoch: 8 	Training Loss: 177.332630
Epoch: 9 	Training Loss: 177.317173
Epoch: 10 	Training Loss: 177.316653
Epoch: 11 	Training Loss: 177.316495
Epoch: 12 	Training Loss: 177.335347
Epoch: 13 	Training Loss: 177.310315
Epoch: 14 	Training Loss: 177.330020
Epoch: 15 	Training Loss: 177.322293
Epoch: 16 	Training Loss: 177.315175
Epoch: 17 	Training Loss: 177.299320
Epoch: 18 	Training Loss: 177.325958
Epoch: 19 	Training Loss: 177.286920
Epoch: 20 	Training Loss: 177.314227
Epoch: 21 	Training Loss: 177.322816
Epoch: 22 	Training Loss: 177.314307
Epoch: 23 	Training Loss: 177.308832
Epoch: 24 	Training Loss: 177.323198
Epoch: 25 	Training Loss: 177.307533
Epoch: 26 	Training Loss: 177.299835
Epoch: 27 	Training Loss: 177.307041
Epoch: 28 

KeyboardInterrupt: ignored

In [8]:
for batch_idx, (data, target) in enumerate(train_loader):
    data = data.unsqueeze(1).type('torch.FloatTensor')
    target = target.type('torch.FloatTensor')
    optimizer.zero_grad()
    output = model(data)
    print("output", output)
    print("target", target)
        

output tensor([[[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5178]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

        [[0.5049]],

      